<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Using Python package

In this notebook, we will present how to use a more advanced python package called [SciPy](https://scipy.org/) to solve the MDA and MDO problem. The same models and the same implementation as in [notebook n°2](02_pure_Python.ipynb) will be used, they thus won't be re-explained here.

## Solving the MDA 

In order to solve the MDA, we will used the `fsolve` function from the `scipy.optimize` package (documentation is available [here](https://docs.scipy.org/doc/scipy/reference/optimize.html)). The [`fsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html) function will solve the equation f(x)=0 so we will need to adjust the implementation. As for the dichotomy method, we will define the function that computes, for a given **MTOW**, the difference between the updated **MTOW** and the actual **MTOW** and feed it to the `fsolve` function.

Let's start by redefining the input parameters for the MDA.

In [ ]:
# Let's start by defining the characteristics related to ...

# ... the geometry
wing_loading = 115.0  # in kg/m2
aspect_ratio = 10.0  # no unit

# ... the target mission
cruise_altitude = 2500.0  # in m
cruise_speed = 80.0  # in m/s
mission_range = 1389000.0  # in m, 750 nm
payload = 320.0  # in kg, corresponds to 4 passengers of 80 kg

# ... the propulsion technology
tsfc = 7.3e-6  # in kg/N/s

Let's now write the function that we want to solve. As a reminder, when this function is equal to zero, the aircraft is converged.

In [ ]:
from modules.pure_python.mtow_loop import mtow_loop


def MTOW_to_solve(
    mtow, aspect_ratio, wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc
):

    MTOW_diff = (
        mtow_loop(
            mtow,
            aspect_ratio,
            wing_loading,
            cruise_altitude,
            cruise_speed,
            mission_range,
            payload,
            tsfc,
        )
        - mtow
    )

    return MTOW_diff

The reader will notice that, even if we only want to solve for the **MTOW**, we still have to pass every constant argument that the `mtow_loop` functions needs. Luckily `fsolve` allows to pass  those arguments for the resolution of the problem. Let's then find the **MTOW** that gives us a converged aircraft !

In [ ]:
from scipy.optimize import fsolve
import numpy as np

# Let's use fsolve in the most simple way possible
initial_guess = 500.0

MTOW_result = fsolve(
    MTOW_to_solve,  # Giving the name of the function we want to solve
    initial_guess,  # The initial guess for the solution
    args=(
        aspect_ratio,
        wing_loading,
        cruise_altitude,
        cruise_speed,
        mission_range,
        payload,
        tsfc,
    ),  # The arguments that won't vary during the problem
    xtol=0.01,  # The relative error between two iterations of the problem
)

print("Result for the MTOW :", float(np.round(MTOW_result, 1)), "kg")

The result is identical to the one obtained in the previous notebook. The slight margin can be explained by the difference in error definition. The first method used an absolute difference, here because of `fsolve`, we use a relative error. 

Let's now move on to the MDO problem.

## MDO solving

As in the first notebook, we will here try to find the aspect ratio that gives the least fuel consumed on the design mission. To do that, we will use another functionnality of `scipy.optimize` called [`minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html). This function can find the minimum of a function with respect to its design variables. Since we want to minimize the fuel consumption based on the aspect ratio, we will give it a function that outputs the fuel consumed on the mission and which has the aspect ratio as the design variable. 

The issue, however, is that **MTOW** will also vary with aspect ratio. Thus, we will need to resize the aircraft for each **AR**. For the sake of this exercise, this can be seen as a constraint to our optimization. So, as an added constraint to our problem, we will tell the `minimize` function that it can modify the aspect ratio but only under the condition that `MTOW_to_solve` is equal to zero, meaning we have a converged aircraft.

All in all, there will be two design variables for this optimization which are going to be the **MTOW** and the **AR**. The constraint will then ensure that the fuel consumption found corresponds to that of a sized aircraft.

At this point, a couple problem arises from the formalism imposed by the `minimize` function:
* The minimize function only works for objective functions that return a single scalar output. This means that we won't be able to reuse `mtow_and_fuel_loop`
* Since we want to ensure that the MTOW is converged, there will be two functions of interest for the `minimize` function. This imposes that both function have to have the same inputs.
* `minimize` can only take a single object as optimization variable. It can however be a tuple or an array when the function takes in reality more than one design variable. Their Python implementation, though, will need to use only a single variable. Consequently, we will need to reformat our functions so that it takes a tuple - which will contain the **MTOW** and the aspect ratio - as input and optimize the function using this tuple as design variable.

The implementation of the function that returns the fuel consumed can go two ways. Either we reuse the `mtow_and_fuel_loop` that we modify to accept the right entry, or, we create a function that runs the performance model with all the other models needed beforehand (geometry for wing area, aerodynamics for lift-to-drag-ratio and weight for **OWE**). The latter solution was chosen and was implemented in a function called `compute_fuel_scipy` located in the [compute_fuel_scipy.py file](modules/python_for_scipy/compute_fuel_scipy.py).

The reformatting of the `mtow_loop` implementation is done in the [mtow_loop_scipy.py file](modules/python_for_scipy/mtow_loop_scipy.py). As with the original `mtow_loop` function, we will then wrap it in a function that returns 0 when the aircraft is converged. This is done in the cell below.

In [ ]:
from modules.python_for_scipy.mtow_loop_scipy import mtow_loop_scipy


def MTOW_to_solve_scipy(
    x, wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc
):

    MTOW_diff = (
        mtow_loop_scipy(
            x,
            wing_loading,
            cruise_altitude,
            cruise_speed,
            mission_range,
            payload,
            tsfc,
        )
        - x[0]
    )

    return MTOW_diff

<div class="alert alert-block alert-info">
<b>Remark</b>
    
We won't be able to use this function to solve the simple MDA problem previously defined. Indeed, since our design variable now consists of the **MTOW** and the aspect ratio, we cannot treat the aspect ratio as an independent constant argument anymore. 
</div>

Before moving on to the minimization of the fuel consumption, let's do a sanity check and ensure that the solution we found earlier is still valid with the new function.

In [ ]:
# The print should return 0
x = (1065.8, 10.0)
result = MTOW_to_solve_scipy(
    x, wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc
)
print(
    "The results of a simple evaluation of the MTOW_to_solve_scipy is ", np.round(result, 1), "kg"
)

Let's now redefine our optimization using the new implementation of the method and see what results it yields.

In [ ]:
from scipy.optimize import minimize
from modules.python_for_scipy.compute_fuel_scipy import compute_fuel_scipy

# Let's start by defining an initial guess of the couple (MTOW, aspect_ratio), tough we now the solution, and
# could thus accelerate the process, we will start at a different value to test the function.
x0 = (500.0, 5.0)

# We can now define the bound of the design variables of the optimization problem. Essentially there
# are two constraints : both value must be positive, but to avoid looking for unfeasible value, we will
# take the same bounds as the arrays previously used for solving the problems
bnds = ((500.0, 5000.0), (1.0, 20.0))

# We can also define the argument of the function
arguments = (wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc)

# Let's now define the constraints, MTOW_to_solve is equal to 0
cons = {"type": "eq", "fun": MTOW_to_solve_scipy, "args": arguments}

# We can now finally call the minimizer
res = minimize(
    compute_fuel_scipy, x0, method="SLSQP", bounds=bnds, constraints=cons, args=arguments
)

print("Number of iterations necessary to find the minimum: ", res.nit)
print("The optimized fuel consumption is equal to ", np.round(res.fun, 4), "kg")
print("The corresponding aspect ratio is :", np.round(res.x[1], 4))
print("The corresponding MTOW is :", np.round(res.x[0], 4), "kg")

The results we get are very close to the one we obtained with our optimization "by hand", both in term of the value of the objective and in term of value of the design variables. Only that this time, the accuracy of the results is not limited by the space between two elements in AR_array. We also managed to find solution with less iterations. Only 22 were necessary here, whereas we had 100 elements in AR_array, which is a considerable step up.

The conclusion, however, remains pretty much the same as in the first notebook. We have to be careful of unit conversions. Besides, data management will only get harder with more modules and the structure of the code is still rigid. For instance, we cannot easily find the optimum (aspect ratio, cruise speed) couple. In order to do so, we would need to change again the syntax of the function to optimize and the constraint function.